# Данные переданные заказчиком

In [2]:
import numpy as np
import re
import pandas as pd
import calendar
from catboost import CatBoostClassifier

ai_sym =['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'D', 'А', 'В', 'Е', 'К', 'М', 'Н', 'О', 'Р', 'С', 'Т', 'У', 'Х']
old_cyrillic_sym = ['И', 'Й', 'Ц', 'Г', 'Ш', 'Щ', 'З', 'Ъ', 'Ф', 'Ы', 'П', 'Л', 'Д', 'Ж', 'Э', 'Я', 'Ч', 'Ь', 'Б', 'Ю']
lat_sym = ['Q', 'W', 'R', 'Y', 'U', 'I', 'S', 'F', 'G', 'J', 'L', 'Z', 'V', 'N']

def d(alist, blist):
    zipped = (zip(alist, blist))
    um1 = dict(sorted(zipped, key = lambda t: t[1], reverse = True))
    return um1

def get_ai_syms(nn_regno, nn_sym_scores):     
    letters = d(nn_regno, nn_sym_scores)
    for s in ai_sym:
        if s not in letters:
            letters[s] = 1    
    return letters

def regno_category(regno):
      
    
    if re.fullmatch('^[А-Я]\d{3}[А-Я]{2}\d{2}(|\d)$', regno) : return 'private car'
    elif re.fullmatch('^[А-Я]{2}\d{5}(|\d)$', regno)  : return ' lk_taxi_r_pricep'
    elif re.fullmatch('^[А-Я]{2}\d{6}(|\d)$', regno)  : return 'lk_pricep_r_transit'
    elif re.fullmatch('^\d{4}[А-Я]{2}\d{2}(|\d)$', regno)  : return 'lk_moto_tract'
    elif re.fullmatch('^[А-Я]{1}\d{4}[А-Я]{2}$', regno) : return 'some'
    elif re.fullmatch('^[А-Я]{2}\d{3}[А-Я]\d{2}(|\d)$', regno)  : return 'lk_transit'
    elif re.fullmatch('^(T|Т)[А-Я]{2}\d{5}(|\d)$', regno) : return 'lk_export'
    elif re.fullmatch('^[А-Я]\d{6}(|\d)$', regno) : return 'lk_mvd_avto'
    elif re.fullmatch('^\d{4}[А-Я]\d{2}(|\d)$', regno) : return 'lk_mvd_moto'
    elif re.fullmatch('^\d{3}[А-Я]\d{2}(|\d)$', regno) : return 'lk_mvd_pricep'
    elif re.fullmatch('^\d{3}(CD|СD|D|T|Т)\d{3}(|\d)(|\d)(|\d)$', regno) : return 'lk_diplomat'
    elif re.fullmatch('^\d{4}[А-Я]{2}\d{2}(|\d)$', regno): return 'lk_army'
    else: return 'unknown'
    
    
def count_foreign_syms(regno):
    i = 0
    for s in regno:
        if s not in (ai_sym + old_cyrillic_sym + lat_sym):
            i += 1

def str_to_list(s):
    if s != '[]':
        s = s[1:-1].split(',')
        return [float(i) for i in s]
    else:
        return [0]

def pick_regno(camera_regno, nn_regno, camera_score, nn_score, nn_sym_scores, nn_len_scores, camera_type, camera_class, 
         time_check, direction, model_name):
    # model_name - path to the model

    x = {}
    x['afts_regno_ai_score'] = nn_score
    x['direction'] = direction
    x['recognition_accuracy'] = camera_score

    nn_sym_scores = str_to_list(nn_sym_scores)
    nn_len_scores = str_to_list(nn_len_scores)
    
    x['max_sym_score'] = np.max(nn_sym_scores)
    x['min_sym_score'] = np.min(nn_sym_scores)
    x['max_len_score'] = np.max(nn_len_scores)
    
    x['ai_len'] = len(nn_regno)
    x['cam_len'] = len(camera_regno)
    
    x['regno_recognize_text'] = ' '.join(camera_regno)
    x['afts_regno_ai_text'] = ' '.join(nn_regno)
    
    x['camera_type'] = camera_type
    x['camera_class'] = camera_class
        
    time_check = pd.to_datetime(time_check)
    x['weekday'] = calendar.day_name[time_check.dayofweek]
    x['month'] = calendar.month_name[time_check.month]
    x['hour'] = time_check.hour
    
    x['regno_template'] = regno_category(camera_regno)
    x['regno_template_ai'] = regno_category(nn_regno)


    x['foreign_sym'] = count_foreign_syms(camera_regno)
    #print(x)
    
    
    letters = get_ai_syms(nn_regno, nn_sym_scores)
    
    
    print(letters)
    x.update(letters)
    
    d = 
    model = CatBoostClassifier()
    model.load_model(model_name)
    print()
    print(sorted(model.feature_names_))
    print()
    print(pd.Series(x))
    y = model.predict_proba(pd.Series(x)[model.feature_names_])
    
    return y
df  = pd.read_csv('test_data.csv')
pick_regno(*df.iloc[5].to_dict().values(), 'micromodel.cbm')


{'1': 0.9999861717224121, '6': 0.9999839067459106, 'А': 0.9999557733535767, 'В': 0.9999555349349976, 'К': 0.9999505281448364, '0': 0.999932050704956, '7': 0.5511531829833984, '2': 1, '3': 1, '4': 1, '5': 1, '8': 1, '9': 1, 'D': 1, 'Е': 1, 'М': 1, 'Н': 1, 'О': 1, 'Р': 1, 'С': 1, 'Т': 1, 'У': 1, 'Х': 1}

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'D', 'afts_regno_ai_score', 'afts_regno_ai_text', 'ai_len', 'cam_len', 'camera_class', 'camera_type', 'direction', 'foreign_sym', 'hour', 'max_len_score', 'max_sym_score', 'min_sym_score', 'month', 'recognition_accuracy', 'regno_recognize_text', 'regno_template', 'regno_template_ai', 'weekday', 'А', 'В', 'Е', 'К', 'М', 'Н', 'О', 'Р', 'С', 'Т', 'У', 'Х']

afts_regno_ai_score              0.549513
direction                               0
recognition_accuracy                 99.0
max_sym_score                    0.999986
min_sym_score                    0.551153
max_len_score                    0.999994
ai_len                              

array([0.43860729, 0.56139271])

In [8]:
print(*df.iloc[5].to_dict().values())


А016ВК797 А016ВК777 99.0 0.5495133996009827 [0.9999557733535767, 0.999932050704956, 0.9999861717224121, 0.9999839067459106, 0.9999555349349976, 0.9999505281448364, 0.9979444146156311, 0.5511531829833984, 0.9993207454681396] [3.7013840881172655e-09, 3.725195485415611e-09, 3.7196645763515335e-09, 3.742908205595086e-09, 3.7187850576714254e-09, 3.7050780221647983e-09, 3.727455677449143e-09, 8.925088224032152e-09, 6.323071374936262e-06, 0.9999936819076538, 3.712293361601837e-09] Стационарная Стрелка-М 2021-08-01 16:18:32 0


In [ ]:
payload = json.dumps({
  "regno_recognize": "А939НО196",
  "afts_regno_ai": "А939НО190",
  "recognition_accuracy": 6.4,
  "afts_regno_ai_score": 0.8689166903495789,
  "afts_regno_ai_char_scores": "[0.9998925924301147, 0.9999872446060181, 0.9999798536300659, 0.9999990463256836, 0.9988356232643127, 0.9998175501823425, 1.0, 0.999994158744812, 0.8702163696289062]",
  "afts_regno_ai_length_scores": "[3.2404470773350624e-10, 3.236617363011618e-10, 3.2367283853140805e-10, 3.2651523151905337e-10, 3.234087164738497e-10, 3.259402747701756e-10, 3.2362224011706076e-10, 4.545459564297971e-09, 2.996458192683349e-08, 1.0, 3.2479344214131345e-10]",
  "camera_type": "Стационарная",
  "camera_class": "Астра-Трафик",
  "time_check": "2021-08-01 09:02:59",
  "direction": 0
})
headers = {
  'Content-Type': 'application/json',
  'x-api-key': '123321'
}

"""Для применения функции повторного запроса требуется реализовать собственный транспортный адаптер."""
adapter = HTTPAdapter(max_retries=3)
session = requests.Session()
session.mount(url, adapter)

In [34]:
df['regno_recognize'].apply(regno_category).unique()
d = df.iloc[0].to_dict()
import json
d = json.JSONEncoder().encode(d)
json.JSONDecoder().decode(d)
#print(json.JSONDecoder(d))

{'regno_recognize': 'А939НО196',
 'afts_regno_ai': 'А939НО190',
 'recognition_accuracy': 6.4,
 'afts_regno_ai_score': 0.8689166903495789,
 'afts_regno_ai_char_scores': '[0.9998925924301147, 0.9999872446060181, 0.9999798536300659, 0.9999990463256836, 0.9988356232643127, 0.9998175501823425, 1.0, 0.999994158744812, 0.8702163696289062]',
 'afts_regno_ai_length_scores': '[3.2404470773350624e-10, 3.236617363011618e-10, 3.2367283853140805e-10, 3.2651523151905337e-10, 3.234087164738497e-10, 3.259402747701756e-10, 3.2362224011706076e-10, 4.545459564297971e-09, 2.996458192683349e-08, 1.0, 3.2479344214131345e-10]',
 'camera_type': 'Стационарная',
 'camera_class': 'Астра-Трафик',
 'time_check': '2021-08-01 09:02:59',
 'direction': 0}

In [23]:
'{%s}' % ', '.join(['"%s": "%s"' % (k, v) for k, v in d.items()])

'{"regno_recognize": "А939НО196", "afts_regno_ai": "А939НО190", "recognition_accuracy": "6.4", "afts_regno_ai_score": "0.8689166903495789", "afts_regno_ai_char_scores": "[0.9998925924301147, 0.9999872446060181, 0.9999798536300659, 0.9999990463256836, 0.9988356232643127, 0.9998175501823425, 1.0, 0.999994158744812, 0.8702163696289062]", "afts_regno_ai_length_scores": "[3.2404470773350624e-10, 3.236617363011618e-10, 3.2367283853140805e-10, 3.2651523151905337e-10, 3.234087164738497e-10, 3.259402747701756e-10, 3.2362224011706076e-10, 4.545459564297971e-09, 2.996458192683349e-08, 1.0, 3.2479344214131345e-10]", "camera_type": "Стационарная", "camera_class": "Астра-Трафик", "time_check": "2021-08-01 09:02:59", "direction": "0"}'